In [50]:
!git clone https://gist.github.com/c1a8c0359fbde2f6dcb92065b8ffc5e3.git

fatal: destination path 'c1a8c0359fbde2f6dcb92065b8ffc5e3' already exists and is not an empty directory.


In [51]:
import pandas

headlines_df = pandas.read_csv('c1a8c0359fbde2f6dcb92065b8ffc5e3/300_stock_headlines.csv')
headlines_df.head(5)

,Unnamed: 0,Unnamed: 0.1,headline,url,publisher,date,stock
0,0,2,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01 00:00:00,A
1,1,3,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18 00:00:00,A
2,2,4,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15 00:00:00,A
3,3,5,"Pershing Square Capital Management, L.P. Buys ...",http://www.gurufocus.com/news/1138704/pershing...,GuruFocus,2020-05-15 00:00:00,A
4,4,6,Agilent Awards Trilogy Sciences with a Golden ...,http://www.gurufocus.com/news/1134012/agilent-...,GuruFocus,2020-05-12 00:00:00,A


In [52]:
import numpy as np
headlines_array = np.array(headlines_df)
np.random.shuffle(headlines_array)
headlines_pre_list = list(headlines_array[:,2])
headlines_list = [s.lower() for s in headlines_pre_list]

print(headlines_list)

['why is agilent (a) up 5.3% since last earnings report?', 'stocks with rising relative strength: agilent technologies', 'j.p. morgan asset management announces liquidation of six exchange-traded funds', "tracking william von mueffling's cantillon capital management portfolio - q4 2018 update", 'kraneshares joins with nasdaq dorsey wright to launch kraneshares china rotation model', 'when does market timing actually work? - november 01, 2019', 'krueger & catalano capital partners, llc buys bristol-myers squibb company, agilent ...', 'synaptics (syna) q3 earnings: will the stock disappoint?', 'hedge funds have never been this bullish on xilinx, inc.', 'top ranked momentum stocks to buy for february 25th', 'stocks to watch: eyes on cyber monday, argentina and mars', "notable earnings after monday's close", 'dividend increases: november 18-22, 2019', "napier park global capital llc buys invesco senior loan, agilent technologies inc, lowe's inc, ...", 'live nation entertainment, zebra tech

In [53]:
!pip install transformers

#Getting the tokenizer and the model

from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")

model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
#That's where the headlines get tokenized to be inputted into model

inputs = tokenizer(headlines_list, padding = True, truncation = True, return_tensors='pt')
print(inputs)

{'input_ids': tensor([[  101,  2339,  2003,  ...,     0,     0,     0],
        [  101, 15768,  2007,  ...,     0,     0,     0],
        [  101,  1046,  1012,  ...,     0,     0,     0],
        ...,
        [  101,  1038,  5302,  ...,     0,     0,     0],
        [  101, 29003,  3372,  ...,     0,     0,     0],
        [  101, 11968, 11649,  ...,     0,     0,     0]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])}


In [ ]:
#inference

outputs = model(**inputs)
print(outputs.logits.shape)

In [ ]:
#Postprocessing with softmax

import torch

predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)

In [ ]:
#Model classes

model.config.id2label

In [ ]:
#Formatting the results as a pandas data frame

import pandas as pd

#Headline #Positive #Negative #Neutral
positive = predictions[:, 0].tolist()
negative = predictions[:, 1].tolist()
neutral = predictions[:, 2].tolist()

table = {'Headline':headlines_list,
         "Positive":positive,
         "Negative":negative, 
         "Neutral":neutral}
      
df = pd.DataFrame(table, columns = ["Headline", "Positive", "Negative", "Neutral"])

#df.head(2990)
#finds the stock to get 
try:
  ticker = str(input("What stock do you want to choose to check if stock go up or down? "))
  target_stock_articles = df[df['Headline'].str.contains(ticker.lower())]

  # calculate the overall sentiment score for each article
  target_stock_articles.loc[:, 'SentimentScore'] = target_stock_articles['Positive'] - target_stock_articles['Negative']

  # calculate the overall sentiment score for all articles relevant to the target stock
  target_stock_sentiment_score = target_stock_articles['SentimentScore'].sum()
  threshold = 0.2
  # make a prediction based on the overall sentiment score
  if target_stock_sentiment_score > threshold:
      print(f"{ticker} stock is predicted to go up.")
  else:
      print(f"{ticker} target stock is predicted to go down.")
except Exception as e:
  print("Error occured")